In [1]:
import pandas as pd
from functions.pred import *
from functions.xai import *
from functions.eval import *
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

c:\Users\ychaf\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
qsts_df = pd.read_csv("../adala.justice.gov/generated_questions.csv")

In [3]:
torch_device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
hf_device = 0 if torch.cuda.is_available() else -1  # only for transformers pipeline()

In [4]:
device = 0 if torch.cuda.is_available() else -1
model_name = "ychafiqui/moroccan_law_qst_classif_arabert2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(torch_device)
model.eval()
print()
# pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [5]:
lime_explainer = LimeExplainer(tokenizer, model, torch_device)
shap_explainer = ShapExplainer(tokenizer, model, hf_device, max_evals=50)  # pipeline wants 0/-1
ig_explainer   = IgExplainer(tokenizer, model, torch_device)

Device set to use cuda:0


In [6]:
train, test = train_test_split(qsts_df, test_size=0.2, random_state=0)

In [7]:
xai_exec = test[["generated_question", "merged_category"]]
xai_exec

,generated_question,merged_category
398,- إذا تقاعد شخص من لدن مؤسسة النظام الجماعي لم...,المادة الأسرية والشؤون الاجتماعية
3833,عقد يتضمن شرطاً فاسخاً يجعل العقد كأن لم يكن، ...,المادة المدنية والالتزامات
4836,"هل أتعرض لغرامة تتراوح بين 30,000 و50,000 درهم...",نصوص عامة
4572,إذا تم إعلان حالة الطوارئ الصحية بموجب مرسوم ص...,نصوص عامة
636,أنا مهندس معماري أعمل بموجب نظام الاستشارة الم...,المادة الإدارية والمالية
...,...,...
4228,أملك محلًا مهنيًا مغلقًا، وبانتهاء الأجل يجب إ...,مادة المهن والتنظيمات المهنية
2367,شخص أجهض امرأة حبلى أو حاول إجهاضها باستخدام ط...,المادة الجنائية
788,إذا صدر قرار من مجلس جماعة ترابية وفقاً للمادة...,المادة الإدارية والمالية
1452,أنا أعتزم التقدم بطلب امتياز لاستغلال تحلية مي...,المادة البيئية والصحية


In [8]:
# def remove_chaklash(text):
#     chaklash_chars = ['ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ']
#     for char in chaklash_chars:
#         text = text.replace(char, '')
#     return text
# xai_exec["generated_question"] = xai_exec["generated_question"].apply(remove_chaklash)

In [ ]:
# xai_exec = xai_exec.sample(10)

In [10]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [11]:
xai_exec[["LIME", "LIME_time"]] = xai_exec.apply(lambda x: lime_explainer.explain(x["generated_question"], x["merged_category"], num_samples=50), axis=1, result_type="expand")

In [12]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [13]:
xai_exec[["SHAP", "SHAP_time"]] = xai_exec.apply(lambda x: shap_explainer.explain(x["generated_question"], x["merged_category"]), axis=1, result_type="expand")

  0%|          | 0/48 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
PartitionExplainer explainer: 2it [00:25, 25.34s/it]               


  0%|          | 0/48 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:16, 16.52s/it]               


  0%|          | 0/48 [00:00<?, ?it/s]

...

PartitionExplainer explainer: 2it [00:12, 12.44s/it]               


  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:11, 11.55s/it]               


  0%|          | 0/48 [00:00<?, ?it/s]

In [14]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [15]:
xai_exec[["IG", "IG_time"]] = xai_exec.apply(lambda x: ig_explainer.explain(x["generated_question"], x["merged_category"]), axis=1, result_type="expand")

In [16]:
xai_exec.to_csv("xai_exec.csv", index=False)